# 1. 损失函数

## 1.1 从数据中学习

在感知机的例子中，我们对照着真值表，人工设定了参数的值，但是那时的参数只有 3 个。而在实际的神经网络中，参数的数量成千上万，在层数更深的深度学习中，参数的数量甚至可以上亿，想要人工决定这些参数的值是不可能的。

神经网络的特征就是可以从数据中学习。所谓“从数据中学习”，是指可以由数据自动决定权重参数的值。

对于图像识别问题，

- 一种方案是，先从图像中提取特征量，再用机器学习技术学习这些特征量的模式。这里所说的“特征量”是指从输入数据（输入图像）中准确地提取的本质数据。

      在计算机视觉领域，常用的特征量包括 SIFT、SURF 和 HOG 等。使用这些特征量将图像数据转换为向量，然后对转换后的向量使用机器学习中的 SVM、KNN 等分类器进行学习。但是需要注意的是，将图像转换为向量时使用的特征量仍是人为设计的。对于不同的问题，必须使用合适的特征量（必须设计专门的特征量），才能得到好的结果。例如，为了区分狗的脸部，人们需要考虑与用于识别手写数字的特征量不同的其它特征量。
      
- 另一种方案是使用神经网络

      神经网络或深度学习则比以往的机器学习方法更能避免人为介入。神经网络的优点是对所有的问题都可以用同样的流程来解决。比如，不管要求解的问题是识别数字，还是识别狗，抑或是识别人脸，神经网络都是通过不断地学习所提供的数据，尝试发现待求解的问题的模式。


![img](image/10.learning.png)

## 1.2 训练和学习

机器学习中，一般将数据分为 **训练数据** 和 **测试数据** 两部分来进行学习和实验等。

首先，使用训练数据进行学习，寻找最优的参数；然后，使用测试数据评价训练得到的模型的实际能力。为什么需要将数据分为训练数据和测试数据呢？因为我们追求的是模型的泛化能力。为了正确评价模型的泛化能力，就必须划分训练数据和测试数据。

**泛化能力** 是指处理未被观察过的数据（不包含在训练数据中的数据）的能力。获得泛化能力是机器学习的最终目标。

例如，在识别手写数字的问题中，泛化能力可能会被用在自动识别银行支票手写数字的系统上。此时，手写数字识别就必须具备较高的识别“某个人”写的数字的能力。注意这里不是“特定的某个人写的特定的数字”，而是“任意一个人写的任意数字”。如果系统只能正确识别已有的训练数据，那有可能是只学习到了训练数据中的个人的习惯写法。

因此，仅仅用一个数据集去学习和评价参数，是无法进行正确评价的。这样会导致可以顺利地处理某个数据集，但无法处理其他数据集的情况，这个状态称为过拟合（over fitting）。

## 1.3 损失函数

神经网络的学习中所用的指标称为损失函数（loss function）。这个损失函数可以使用任意函数，但一般用均方误差和交叉熵误差等。

### 均方误差

可以用作损失函数的函数有很多，其中最有名的是均方误差（MSE, mean squared error）。均方误差如下式所示。

$$ E = \frac{1}{2} \sum_k(y_k-t_k)^2 $$

其中，$y_k$ 是表示神经网络的输出，$t_k$ 表示标签数据，$k$ 表示数据的维数。

均方误差会计算神经网络的输出和正确解监督数据的各个元素之差的平方，再求总和。现在，我们用Python来实现这个均方误差，

In [4]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

例如，在手写数字识别的例子中，yk、tk 是由如下 10 个元素构成的数据。

In [7]:
import numpy as np

y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])
mean_squared_error(y, t)

0.09750000000000003

### 交叉熵误差

除了均方误差之外，交叉熵误差（cross entropy error）也经常被用作损失函数。交叉熵误差如下式所示。

$$ E = -\sum_kt_klogy_k $$

因为$t_k$中只有正确解标签的索引为 1，其他均为 0（one-hot 表示）。因此，交叉熵实际上只计算对应正确解标签的输出的自然对数。

我们来用代码实现交叉熵误差：

In [9]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

函数内部在计算 np.log 时，加上了一个微小值 delta。这是因为，当出现 np.log(0) 时，np.log(0) 会变为负无限大的 -inf，这样一来就会导致后续计算无法进行。作为保护性对策，添加一个微小值可以防止负无限大的发生。

In [10]:
np.log(0)

C:\Users\Devon\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


-inf

In [12]:
cross_entropy_error(y, t)

0.510825457099338

## 1.4 损失函数的作用

在神经网络的学习中，寻找最优参数（权重和偏置）时，要寻找使损失函数的值尽可能小的参数。

# 2. 反向传播算法